In [37]:
import pandas as pd
import re

In [38]:
df=pd.read_csv('data_content_based.csv')
df=df.iloc[0:30000]
df = df.astype({'id':'int16'})
df = df.astype({'tags':'string'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      30000 non-null  int16 
 1   tags    30000 non-null  string
dtypes: int16(1), string(1)
memory usage: 293.1 KB


In [25]:
df.head()

<bound method DataFrame.info of           id                                               tags
0      10000  palm tree girl sp jace sko white skirtssp jace...
1      10001  palm tree kid girl sp jema skt blue skirtssp j...
2      10002  palm tree kid sp jema skt blue skirtssp jema s...
3      10003  nike women as nike elem white t-shirta nike el...
4      10004  nike men as 7 sw temp grey shortsa 7 sw tempad...
...      ...                                                ...
38628   9995  gini and joni girl su scotia sko pink skirtssu...
38629   9996  palm tree girl sp jelli top blue topssp jelli ...
38630   9997  palm tree girl hs livia top blue topsh livia t...
38631   9998  palm tree girl hs livia top pink topsh livia t...
38632   9999  palm tree kid girl mm merlin top blue topsmm m...

[38633 rows x 2 columns]>

In [39]:
def preprocess_text(text):
    text.lower()
    text=re.sub(r'\d+','',text)
    return text

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
cv= CountVectorizer(max_features=1000,stop_words='english',preprocessor=preprocess_text)
vectors=cv.fit_transform(df['tags']).toarray()
vectors = vectors.astype('int8')
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [29]:
cv.get_feature_names()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['abov',
 'abras',
 'abrasion',
 'absolut',
 'absorb',
 'absorpt',
 'accents',
 'accentu',
 'accessori',
 'accessories',
 'accessory',
 'accommod',
 'activ',
 'ad',
 'add',
 'addit',
 'adida',
 'adidas',
 'adjust',
 'adults',
 'aerodynam',
 'air',
 'allow',
 'ampl',
 'analog',
 'analogu',
 'aneri',
 'ani',
 'ankl',
 'apertur',
 'appeal',
 'appliqu',
 'appliqué',
 'appliquéd',
 'arch',
 'area',
 'arm',
 'arrow',
 'art',
 'assn',
 'attach',
 'aurelia',
 'avail',
 'avoid',
 'away',
 'backdial',
 'backfoot',
 'backpack',
 'badg',
 'bag',
 'baggit',
 'balanc',
 'ban',
 'band',
 'base',
 'basic',
 'basketbal',
 'beach',
 'bead',
 'becom',
 'befor',
 'beig',
 'belt',
 'benetton',
 'best',
 'better',
 'bezel',
 'biba',
 'big',
 'bikini',
 'black',
 'blazer',
 'bleach',
 'bleachdri',
 'bleachtumbl',
 'blend',
 'block',
 'blue',
 'bluecomposit',
 'boat',
 'bodi',
 'boho',
 'bold',
 'boot',
 'bow',
 'box',
 'boy',
 'boysboyscasu',
 'bracelet',
 'brand',
 'breathabl',
 'bridg',
 'brief',
 'bright'

In [41]:
from sklearn.metrics.pairwise import cosine_similarity as cs
similarity=cs(vectors)

In [31]:
similarity.dtype

dtype('float64')

In [42]:
similarity = similarity.astype('float16')

In [16]:
def recommend(product):
    rec_list=[]
    product_index=df[df['id']==product].index[0]
    distances=similarity[product_index]
    product_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:100]
    for i in product_list:
        rec_list.append(df.iloc[i[0]].id)
    return rec_list
    

In [17]:
product_info=pd.read_csv('data_combined.csv', index_col=False, dtype='unicode')


In [18]:
product_info.head()

,code,requestId,id,price,discountedPrice,styleType,productTypeId,articleNumber,visualTag,productDisplayName,...,otherFlags,articleDisplayAttr,productDescriptors,styleOptions,discountData,colours,productSpecificationEntry,associatedImageProducts,relatedStyleOptions,styleVideoAlbumList
0,NaN,NaN,10000,649,324,DEL,304,132197135028 1303,NaN,Palm Tree Girls Sp Jace Sko White Skirts,...,"[{'dataType': 'BOOLEAN', 'name': 'isFragile', ...","{'id': 74, 'core': {'order': '0', 'display': '...",{'description': {'descriptorType': 'descriptio...,"[{'id': 39001, 'name': 'Size', 'value': '2YRS'...","{'discountType': 1, 'discountAmount': 325, 'di...",NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,10001,549,274,DEL,304,132215584945 6300,NaN,Palm Tree Kids Girls Sp Jema Skt Blue Skirts,...,"[{'dataType': 'BOOLEAN', 'name': 'isFragile', ...","{'id': 74, 'core': {'order': '0', 'display': '...",{'description': {'descriptorType': 'descriptio...,"[{'id': 39005, 'name': 'Size', 'value': '2YRS'...","{'discountType': 1, 'discountAmount': 275, 'di...",NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,10002,549,274,DEL,304,132215584945 8300,NaN,Palm Tree Kids Sp Jema Skt Blue Skirts,...,"[{'dataType': 'BOOLEAN', 'name': 'isFragile', ...","{'id': 74, 'core': {'order': '0', 'display': '...",{'description': {'descriptorType': 'descriptio...,"[{'id': 39009, 'name': 'Size', 'value': '2YRS'...","{'discountType': 1, 'discountAmount': 275, 'di...",NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,10003,2695,2695,P,289,376668-100,NaN,Nike Women As Nike Eleme White T-Shirt,...,"[{'dataType': 'BOOLEAN', 'name': 'isFragile', ...","{'id': 90, 'core': {'order': '0', 'display': '...",{'description': {'descriptorType': 'descriptio...,"[{'id': 39010, 'name': 'Size', 'value': 'S', '...",NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,10004,1995,1995,P,303,393799-064,NaN,Nike Men As 7 Sw Temp Grey Shorts,...,"[{'dataType': 'BOOLEAN', 'name': 'isFragile', ...","{'id': 72, 'core': {'order': '0', 'display': '...",{'description': {'descriptorType': 'descriptio...,"[{'id': 39014, 'name': 'Size', 'value': 'S', '...",NaN,{'colors': {'22589': {'dre_landing_page_url': ...,NaN,NaN,NaN,NaN


In [19]:
print(recommend(10004))
k=recommend(10004)

[10751, 10020, 15328, 30595, 10021, 30598, 31461, 31449, 31458, 10012, 30596, 30599, 10033, 32648, 31452, 32649, 30610, 31422, 30602, 10024, 11269, 28627, 30606, 31442, 11270, 30609, 28505, 10030, 22590, 10032, 32621, 31448, 28503, 10010, 22604, 28611, 28513, 31466, 31459, 25024, 10019, 17934, 10750, 28640, 19948, 22607, 22644, 30639, 22584, 22589, 28504, 10721, 14021, 28512, 30638, 21593, 14025, 31467, 17896, 28506, 10031, 15609, 15330, 25029, 28628, 10778, 10785, 30636, 15334, 31462, 15331, 15333, 15335, 10786, 25043, 25030, 14022, 28536, 31604, 25038, 31455, 10027, 14024, 31444, 15610, 25042, 15613, 31456, 31447, 31460, 28514, 28515, 12811, 11275, 28622, 10784, 15643, 31451, 31445]


In [20]:
for i in k:
    print(*product_info.loc[product_info['id'] ==str(i)].productDisplayName.values)

Nike Men Breakline Grey Track Pants
Nike Men Black & White Reversible Shorts
Nike Men Solid Blue Sweaters
Nike Men Woven Grey Shorts
Nike Men As Woven Shor White Shorts
Nike Men Hustle Reversible Black Shorts
Nike Men Tennis White Shorts
Nike Men Training Black Shorts
Nike Men Running Grey Shorts
Nike Men's Breakli Navy Blue Track Pants
Nike Men Woven Blue Shorts
Nike Men Hustle Reversible Red Shorts
Nike Men As Ad Breakli Navy Blue Shorts
Nike Black Legacy Woven     Training  Shorts
Nike Men Compression Grey T-shirt
Nike Men Black Shorts
Nike Men Black Shorts
Nike Men White Shorts
Nike Boys Basketball Black Shorts
Nike Women As Make Yours Black T-Shirts
Nike Men Laced Crew Black T-Shirts
Nike Men Longer Knit Black Shorts
Nike Men Running Blue Shorts
Nike Men Cricket Grey Jersey
Nike Men As Ss Cruiser Maroon T-Shirts
Nike Men Black Shorts
Nike Men Tempo Grey Shorts
Nike Women As See The Win Black T-Shirts
Nike Men Solid Grey Shorts
Nike Men AS FCB Core T Black T-shirts
Nike Men Woven R

In [43]:
import pickle

In [ ]:
pickle.dump(similarity,open('content_based_model.pkl','wb'))